# <center><b> APP CONFIGURATION </b></center>

## Why they are used?

### 1. Web
        - Web Content Management Systems and small web apps use them to save credentials (DB or Mail server usernames and passwords)
        - App Configurations (web.xml)
### 2. Scientific Softwares
        - Saving the path to libraries or necessary files
        - Configurations for different experiments
### 3. Local Softwares
        - Games (the herding env)
        - Mail Clients
        - ...
### Alternative methods:
        - They can be hard coded which is an inexpensive and more readble way for small apps or apps with limited use.
        - Using command line arguments. This is great for shell scripts. All the options can be generated by an external program and fed to the main app.

# In Python

Here we cover all the python libraries for this.

1. Hard coding
2. INI files
3. YAML files
4. JSON files
5. XML files
6. Command line (argparse library)

In [1]:
# INSTALLS
!pip -q install configparser bs4 lxml

# The Agent Class

Let's define an agent class and use the loaded configuration to initiate an instance of it.

In [2]:
# Defining an abstract class for our agent
class Agent:
    def __init__(self, config):
        print("Setting up the agent with these\n-------------")
        print("Algorithm: "+ str(config['algorithm']))
        print("Layers: "+ str(config['layer_nodes']))
        print("activations: "+ str(config['activations']))
        print("loss: "+ str(config['loss']))
        print("learning Rate: "+ str(float(config['learning_rate'])))
    
    def action(self):
        pass
    
    def learn(self):
        pass

# 1- Hard Coding
can be done by python dictionaries, a class or python library:

In [3]:
def load_confs():
    agent_conf = {
        "algorithm":"ppo",
        "layer_nodes":[64,64,4],
        "activations":["relu", "relu", ""],
        "loss":"mse",
        "optimizer":"adam",
        "learning_rate":5e-4
    }
    env_conf = {
        "agents":4,
        "fog":True
    }
    confs = {"agent":agent_conf,"env":env_conf}
    return confs

confs = load_confs()
    
agent = Agent(confs["agent"])

Setting up the agent with these
-------------
Algorithm: ppo
Layers: [64, 64, 4]
activations: ['relu', 'relu', '']
loss: mse
learning Rate: 0.0005


# 2- INI Files
are mainly used in windows apps. Thus, the Windows operating system and most softwares develops by and for Windows use this standard. It looks like this:

```bash
[section1]
option1=value1
option2=value2

[section2]
option3=value3
option4=value4
```
In python, the **ConfigParser** package can process these files.

In [4]:
%%file experiment.ini
[agent]
algorithm=ppo
layer_nodes=64,64,4
activations=relu,relu,
loss:mse
optimizer:adam
learning_rate:5e-4

[env]
agents=4
fog=True

Writing experiment.ini


In [5]:
from configparser import ConfigParser

configs = ConfigParser()

# Opening the file
with open('experiment.ini', 'r') as f:
    configs.read_file(f)

# Travesal
for section in configs.sections():
    print("Processing Section: "+section+"\n-------")
    for option in configs.options(section):
        print("Processing "+ option + " -> " + configs.get(section,option))
    print("\n")

layers = configs.get("agent","layer_nodes").split(",")
print("Processing list of values: "+ str(layers))

no_agents = configs.get("env","agents")
print("Number of agents: " + str(no_agents) + ". Its type is " + str(type(no_agents)) +\
      ". Don't forget to cast it before using it.")

Processing Section: agent
-------
Processing algorithm -> ppo
Processing layer_nodes -> 64,64,4
Processing activations -> relu,relu,
Processing loss -> mse
Processing optimizer -> adam
Processing learning_rate -> 5e-4


Processing Section: env
-------
Processing agents -> 4
Processing fog -> True


Processing list of values: ['64', '64', '4']
Number of agents: 4. Its type is <class 'str'>. Don't forget to cast it before using it.


## Loading the INI file and configuring the agent:

In [6]:
def load_confs():
    configs = ConfigParser()

    # Opening the file
    with open('experiment.ini', 'r') as f:
        configs.read_file(f)

    agent_conf = {
        "algorithm":configs.get("agent","algorithm"),
        "layer_nodes": list(map(int, configs.get("agent","layer_nodes").split(","))), #[64,64,4]
        "activations": configs.get("agent","activations").split(","), #["relu", "relu", ""]
        "loss":configs.get("agent","loss"),
        "optimizer":configs.get("agent","optimizer"),
        "learning_rate":configs.get("agent","learning_rate") # 5e-4
    }
    env_conf = {
        "agents":configs.get("env","agents"),
        "fog":bool(configs.get("env","fog"))
    }
    confs = {"agent":agent_conf,"env":env_conf}
    return confs

confs = load_confs()
    
agent = Agent(confs["agent"])

Setting up the agent with these
-------------
Algorithm: ppo
Layers: [64, 64, 4]
activations: ['relu', 'relu', '']
loss: mse
learning Rate: 0.0005


In [7]:
# Cleaning up
!rm experiment.ini

# 2- YAML
YAML (rhymes with camel) is a configuration file popular among programmers especially python developers. As wikipedia put it:
> YAML (a recursive acronym for "YAML Ain't Markup Language") is a human-readable data-serialization language. It is commonly used for configuration files and in applications where data is being stored or transmitted. YAML targets many of the same communications applications as Extensible Markup Language (XML) but has a minimal syntax which intentionally differs from SGML. It uses both Python-style indentation to indicate nesting, and a more compact format that uses [...] for lists and {...} for maps making YAML 1.2 a superset of JSON. The official recommended filename extension for YAML files has been *.yaml* since 2006. [But .yml is more popular]

YAML is JSON's official subset. YAML has been criticized for its large portion of whitespace, and difficult editing.

In [8]:
%%file experiment.yml
agent:
    algorithm: ppo
    layer_nodes: 64,64,4
    activations: relu,relu,
    loss: mse
    optimizer: adam
    learning_rate: 5e-4

env:
    agents: 4
    fog: True

Writing experiment.yml


In [9]:
import yaml

with open("experiment.yml", "r") as ymlfile:
    configs = yaml.safe_load(ymlfile) # used to be yaml.load(ymlfile)

print("Sections are:\n---------")
for section in configs:
    print(section)
print("\n")

print("Each section is a dictionary:\n---------")
print('configs["agent"] = ' + str(configs["agent"]))
print()
print('configs["env"] = ' + str(configs["env"]))
print()

print("Here's the network config which comes as a python array: " + str(configs["agent"]["layer_nodes"]))
nodes = configs["agent"]["layer_nodes"].split(',')
nodes

Sections are:
---------
agent
env


Each section is a dictionary:
---------
configs["agent"] = {'algorithm': 'ppo', 'layer_nodes': '64,64,4', 'activations': 'relu,relu,', 'loss': 'mse', 'optimizer': 'adam', 'learning_rate': '5e-4'}

configs["env"] = {'agents': 4, 'fog': True}

Here's the network config which comes as a python array: 64,64,4


['64', '64', '4']

In [10]:
import yaml

def load_confs():
    # Opening the file
    with open("experiment.yml", "r") as ymlfile:
        configs = yaml.safe_load(ymlfile) # used to be yaml.load(ymlfile)

    agent_conf = {
        "algorithm":configs["agent"]["algorithm"],
        "layer_nodes": list(map(int, configs["agent"]["layer_nodes"].split(','))), #[64,64,4]
        "activations": configs["agent"]["activations"].split(','), #["relu", "relu", ""]
        "loss":configs["agent"]["loss"],
        "optimizer":configs["agent"]["optimizer"],
        "learning_rate":float(configs["agent"]["learning_rate"]) # 5e-4
    }
    env_conf = {
        "agents":configs["env"]["agents"],
        "fog":bool(configs["env"]["fog"])
    }
    confs = {"agent":agent_conf,"env":env_conf}
    return confs

confs = load_confs()
    
agent = Agent(confs["agent"])

Setting up the agent with these
-------------
Algorithm: ppo
Layers: [64, 64, 4]
activations: ['relu', 'relu', '']
loss: mse
learning Rate: 0.0005


In [11]:
!rm experiment.yml

## A popular use of YAML files - Creating Conda venv

[Conda Cheet Sheet](https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf)

With conda, you can create, export, list, remove, and update environments that have different versions of Python and/or packages installed in them. Switching or moving between environments is called activating the environment. You can also share an environment file [\[1\]](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html). By default, environments are installed into the `envs` directory in your conda directory.

After creating, we have to `activate` our environment. In Windows this is done by `activate ENV_NAME` or `deactivate` commands from the command line. In Linux and macOS, we can use `conda/source activate ENV_NAME` or `conda/source deactivate` (source is depricated).

### Some useful commands:

1. To create an environment:
```bash
conda create --name drl
```

2. To create an environment with a specific version of Python:
```bash
conda create -n drl python=3.6
```

3. To create an environment with a specific package:
```bash
conda create -n drl scipy
```
or
```bash
conda create -n drl python
conda install -n drl scipy
```

4. Create the environment from the environment.yml file:
```bash
conda env create -f environment.yml
```

A sample environment file would look like this [\[2\]](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#create-env-file-manually):

```
name: drl
channels:
  - defaults
dependencies:
  - python=3.6.*
  - bz2file=0.98
  - cython=0.29.*
  - pip=19.1.*
  - numpy=1.16.*
  - jupyter=1.0.*
  - matplotlib=3.1.*
  - setuptools=41.0.*
  - scikit-learn=0.21.*
  - scipy=1.2.*
  - pandas=0.24.*
  - pillow=6.1.*
  - seaborn=0.9.*
  - h5py=2.9.*
  - pytest=5.0.*
  - twisted=19.2.*
  - tensorflow=2.0.*
  - tensorflow-gpu=2.0.*
  - pip:
    - keras==2.2.*
```

5. To remove an environment, in your terminal window or an Anaconda Prompt, run:
```bash
conda remove --name drl
```

6. To query the existing environments run:
```bash
conda info --env
```
OR
```bash
conda env list
```

7. Exporting an environment file across platforms. If you want to make your environment file work across platforms, you can use:
```bash
conda env export > environment.yml
```
__<u style="color:red;">Note:</u> Don't forget to remove the generated `environment.yml` last line__

<!--
python ==3.5
ffmpeg=3.2.4
freetype=2.7
imageio=2.2.0
...
Then do:

conda install --yes --file requirements.txt
-->

# 3- JSON
is short of Javascript Object Notation. It's mainly used for data communication (web services) and storage (MongoDB). Due to its wide-spread support, it's also a good option for a configuration file.

In [12]:
%%file experiment.json
{
    "agent": {
    "algorithm": "ppo",
    "network":{
        "layer_nodes":["64","64","4"],
        "activations":["relu", "relu",""],
        "loss":"mse",
        "optimizer":"adam",
        "learning_rate":"5e-4"
        }
    },

    "env": {
        "agents": "4",
        "fog": "true"
    }
}

Writing experiment.json


In [13]:
import json, pprint

with open("experiment.json", "r") as json_config:
    configs = json.load(json_config)

pprint.pprint(configs) # Using Pretty Print to print out a formatted output

print(configs["agent"]["network"]["layer_nodes"])

{'agent': {'algorithm': 'ppo',
           'network': {'activations': ['relu', 'relu', ''],
                       'layer_nodes': ['64', '64', '4'],
                       'learning_rate': '5e-4',
                       'loss': 'mse',
                       'optimizer': 'adam'}},
 'env': {'agents': '4', 'fog': 'true'}}
['64', '64', '4']


In [14]:
import json

def load_confs():
    # Opening the file
    with open("experiment.json", "r") as json_config:
        configs = json.load(json_config)

    agent_conf = {
        "algorithm":configs["agent"]["algorithm"],
        "layer_nodes": list(map(int, configs["agent"]["network"]["layer_nodes"])), #[64,64,4]
        "activations": configs["agent"]["network"]["activations"],  #["relu", "relu", ""]
        "loss":configs["agent"]["network"]["loss"],
        "optimizer":configs["agent"]["network"]["optimizer"],
        "learning_rate":float(configs["agent"]["network"]["learning_rate"]) # 5e-4
    }
    env_conf = {
        "agents":configs["env"]["agents"],
        "fog":bool(configs["env"]["fog"])
    }
    confs = {"agent":agent_conf,"env":env_conf}
    return confs

confs = load_confs()
    
agent = Agent(confs["agent"])

Setting up the agent with these
-------------
Algorithm: ppo
Layers: [64, 64, 4]
activations: ['relu', 'relu', '']
loss: mse
learning Rate: 0.0005


In [15]:
!rm experiment.json

# 4. XML

In [16]:
%%file experiment.xml
<config>
    <agent>
    <algorithm>ppo</algorithm>
        <network>
            <layer_nodes>64,64,4</layer_nodes>
            <activations>relu,relu,</activations>
            <loss>mse</loss>
            <optimizer>adam</optimizer>
            <learning_rate>5e-4</learning_rate>
        </network>
    </agent>

    <env>
        <agents>4</agents>
        <fog>true</fog>
    </env>
</config>

Writing experiment.xml


In [17]:
from bs4 import BeautifulSoup
import lxml

with open("experiment.xml","r") as xml_file:
    contents = xml_file.read()

configs = BeautifulSoup(contents, "lxml")

print(configs.agent.algorithm.contents)
print(configs.agent.network.layer_nodes.contents[0].split(","))
print(configs.env.fog.contents)
print(configs.agent.network.activations.contents[0].split(","))
print(configs.agent.network.optimizer)

configs

['ppo']
['64', '64', '4']
['true']
['relu', 'relu', '']
<optimizer>adam</optimizer>


<html><body><config>
<agent>
<algorithm>ppo</algorithm>
<network>
<layer_nodes>64,64,4</layer_nodes>
<activations>relu,relu,</activations>
<loss>mse</loss>
<optimizer>adam</optimizer>
<learning_rate>5e-4</learning_rate>
</network>
</agent>
<env>
<agents>4</agents>
<fog>true</fog>
</env>
</config>
</body></html>

In [18]:
from bs4 import BeautifulSoup
import lxml

def load_confs():
    # Opening the file
    with open("experiment.xml","r") as xml_file:
        contents = xml_file.read()

    configs = BeautifulSoup(contents, "lxml")
    
    agent_conf = {
        "algorithm":configs.agent.algorithm.contents[0],
        "layer_nodes": list(map(int, configs.agent.network.layer_nodes.contents[0].split(","))), #[64,64,4]
        "activations": configs.agent.network.activations.contents[0].split(","),  #["relu", "relu", ""]
        "loss":configs.agent.network.loss.contents[0],
        "optimizer":configs.agent.network.optimizer.contents[0],
        "learning_rate":float(configs.agent.network.learning_rate.contents[0]) # 5e-4
    }
    env_conf = {
        "agents":int(configs.env.agents.contents[0]),
        "fog":bool(configs.env.fog.contents[0])
    }
    confs = {"agent":agent_conf,"env":env_conf}
    return confs

confs = load_confs()
    
agent = Agent(confs["agent"])

Setting up the agent with these
-------------
Algorithm: ppo
Layers: [64, 64, 4]
activations: ['relu', 'relu', '']
loss: mse
learning Rate: 0.0005


In [19]:
!rm experiment.xml

# Command Line

The argparse module makes it easy to write user-friendly command-line interfaces. The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.

```python
import argparse

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

args = parser.parse_args()
print(args.accumulate(args.integers))
```

Assuming the Python code above is saved into a file called prog.py, it can be run at the command line and provides useful help messages:

```bash
$ python prog.py -h
usage: prog.py [-h] [--sum] N [N ...]
```

When run with the appropriate arguments, it prints either the sum or the max of the command-line integers:

```bash
$ python prog.py 1 2 3 4
4

$ python prog.py 1 2 3 4 --sum
10
```

In [20]:
%%file prog.py
import argparse

parser = argparse.ArgumentParser(description="Read agent's configurations.")
parser.add_argument("--algorithm", help="Specifies type of the algorithm", default="ddpg")
parser.add_argument("--lr", help="Sets the learning rate", default="1e-4")

args = parser.parse_args()
print(args)
print(args.algorithm)
print(args.lr)

Writing prog.py


In [21]:
# Reading from defaults
%run -i prog.py

Namespace(algorithm='ddpg', lr='1e-4')
ddpg
1e-4


In [22]:
# Reading from the command line
%run -i prog.py --algorithm ppo --lr 5e-4

Namespace(algorithm='ppo', lr='5e-4')
ppo
5e-4


In [23]:
%run -i prog.py -h

usage: prog.py [-h] [--algorithm ALGORITHM] [--lr LR]

Read agent's configurations.

optional arguments:
  -h, --help            show this help message and exit
  --algorithm ALGORITHM
                        Specifies type of the algorithm
  --lr LR               Sets the learning rate


In [24]:
!rm prog.py

<a href="https://www.linkedin.com/in/fredamouzgar/"><p style="font-size:8px">&#169; F.A, 2020</p></a>